In [2]:

from gurobipy import Model, GRB, quicksum
import pandas as pd
from code_map import meters, markets
import numpy as np
import matplotlib.pyplot as plt
import calendar 
from datetime import datetime
import pytz
import openpyxl
from itertools import combinations, product


In [3]:
class Portfolio:
    def __init__(self, assets : list[meters.PowerMeter], hour : pd.Timestamp):
        self.assets = assets
        self.hour = hour
        self.response_time = self.get_response_time()
        self.flex_volume_up = self.get_flex_volume_up(hour)
        self.flex_volume_down = self.get_flex_volume_down(hour)
        self.sleep_time = self.get_sleep_time()
    
    def add_asset(self, asset):
        self.assets.append(asset)
        
    def remove_asset(self, asset):
        self.assets.remove(asset)
    
    def get_response_time(self):
        return np.mean([asset.response_time for asset in self.assets])
    
    def get_sleep_time(self):
        return np.mean([asset.sleep_time for asset in self.assets])
    
    def get_flex_volume_up(self, hour : pd.Timestamp):
        return sum([asset.flex_volume["value"].loc[(asset.flex_volume["Time(Local)"] == hour)] for asset in self.assets if asset.direction != "down"])
    
    def get_flex_volume_down(self, hour : pd.Timestamp):
        return sum([asset.flex_volume["value"].loc[(asset.flex_volume["Time(Local)"] == hour)] for asset in self.assets if asset.direction != "up"])

In [4]:
market_list = markets.all_market_list

In [5]:
market_dict = {market.name : market for market in market_list}

In [7]:
timeframe = pd.date_range(start=pd.Timestamp(year= markets.year, month= markets.start_month, day = markets.start_day, hour = markets.start_hour), 
                              end= pd.Timestamp(year = markets.year, month = markets.end_month, day = markets.end_day, hour = markets.end_hour), freq="H", tz = "Europe/Oslo")

In [8]:
market_names = [market.name for market in market_list]

In [9]:
power_meter_dict = meters.power_meters

In [10]:
# make all possible combination of portfolios for each hour and every power-meter and every market
def generate_portfolios(assets : list[meters.PowerMeter], markets : list[markets.ReserveMarket], timestamps):
    portfolios = []

    for hour in timestamps:
        for num_assets in range(len(assets) + 1):
            # Generate all possible combinations of assets for the current hour
            asset_combinations = combinations(assets, num_assets)

            for asset_combination in asset_combinations:
                portfolio = Portfolio(list(asset_combination), hour)
                portfolios.append(portfolio)

    return portfolios

In [11]:
#portfolios = generate_portfolios(list(power_meter_dict.values()), market_list, timeframe)

In [12]:

H = timeframe # set of hours in timehorizone
M = market_list # set of markets

market_per_hour = [market_list] * len(H) # set of markets per hour

P = []  #set of portfolios
L = list(power_meter_dict.values()) # set of meters

load_pr_hour = [L] * len(H) # set of meters per hour




In [13]:
[len(market.price_data) for market in M]

[24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24]

In [14]:
[market.name for market in M]

['FFR_flex',
 'FFR_profile',
 'FCR_D_1_D',
 'FCR_D_1_N',
 'FCR_D_2_D',
 'FCR_D_2_N',
 'aFRR up',
 'aFRR down',
 'RK_up',
 'RK_down',
 'RKOM_H_up',
 'RKOM_H_down',
 'RKOM_B_up',
 'RKOM_B_down']

In [26]:
L_v = [[meter.flex_volume["value"].loc[meter.flex_volume["Time(Local)"] == hour].values[0] for meter in L] for hour in H] # set of flex volumes for meters

L_s = [[meter.sleep_time for meter in L]] * len(H) # set of sleep times for meters
L_r = [[meter.response_time for meter in L]] * len(H) # set of response times for meters

M_p = [[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1] for market in M] for hour in H] # set of prices for markets
M_v = [[market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1] for market in M] for hour in H] # set of volumes for markets

M_m = [[market.min_volume for market in M]] * len(H) # set of min values for markets
M_r = [[market.response_time for market in M]] * len(H) # set of response times for markets
M_s = [[market.sleep_time for market in M]] * len(H) # set of sleep times for markets



In [27]:
M_m

[[5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10],
 [5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10],
 [5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10],
 [5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10],
 [5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10],
 [5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10],
 [5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10],
 [5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10],
 [5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10],
 [5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10],
 [5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10],
 [5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10],
 [5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10],
 [5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10],
 [5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10],
 [5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10],
 [5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10],
 [5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10],
 [5, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10],
 [5, 1, 1, 1, 1, 1, 1, 1, 10, 1

In [ ]:
len(market_per_hour)

In [28]:
portfolio1 = Portfolio(assets = L[:6], hour = timeframe[5])

In [29]:
portfolio1.assets

In [ ]:
portfolio1.sleep_time

In [ ]:
type(L)

In [ ]:
capacity_strings = ["FFR", "FCR", "aFRR", "RKOM"] # set of markets for controllable load
Mc = [m for m in M if any(s in m.name for s in capacity_strings)] # set of capacity markets
Ma = [m for m in M if not any(s in m.name for s in capacity_strings)] # set of activation markets



income = tilgjengelig volum * pris

In [ ]:
def get_portfolio_income(portfolio : Portfolio, market : markets.ReserveMarket, hour : pd.Timestamp):
    
    if market.direction == "up": 
        return portfolio.flex_volume_up * market.price_data.loc[(market.price_data["Time(Local)"] == hour)].values[0]
    elif market.direction == "down":
        return portfolio.flex_volume_down * market.price_data.loc[(market.price_data["Time(Local)"] == hour)].values[0]
    else:
        return max(portfolio.flex_volume_up, portfolio.flex_volume_down) * market.price_data.loc[(market.price_data["Time(Local)"] == hour)].values[0]
        

In [ ]:
def get_possible_arming_income(hour : pd.Timestamp, load : meters.PowerMeter, market : markets.ReserveMarket):
    """ Returns the possible income for a given hour, load and capacity market

    Args:
        hour (pd.Timestamp): _description_
        load (meters.PowerMeter): _description_
        market (markets.ReserveMarket): _description_

    Returns:
        _type_: _description_
    """
    consumption_df = load.consumption_data
    available_flex = consumption_df.loc[(consumption_df["Time(Local)"] == hour)]
    market_df = market.price_data
    volume_df = market.volume_data
    possbile_income = market_df.loc[market_df["Time(Local)"] == hour] if volume_df.loc[volume_df["Time(Local)"] == hour] > 0 else 0
    return available_flex * possbile_income

In [ ]:
def get_possible_rk_income(hour : pd.Timestamp, load : meters.PowerMeter, market : markets.ReserveMarket, direction : str):
    """income for activating portfolio p that was bid into activation market m at hour h
    """
    rk_up_price = market_dict["RK_up"].price_data.loc[market_dict["RK_up"].price_data["Time(Local)"] == hour] if market_dict["RK_up"].volume_data.loc[market_dict["RK_up"].volume_data["Time(Local)"] == hour] > 0 else 0
    rk_down_price = market_dict["RK_down"].price_data.loc[market_dict["RK_down"].price_data["Time(Local)"] == hour] if market_dict["RK_down"].volume_data.loc[market_dict["RK_down"].volume_data["Time(Local)"] == hour] > 0 else 0
       
    if direction == "up":
            return rk_up_price * load.consumption_data.loc[load.consumption_data["Time(Local)"] == hour]
    elif direction == "down":
        return rk_down_price * load.consumption_data.loc[load.consumption_data["Time(Local)"] == hour]
    else:
        return max(rk_up_price, rk_down_price) * load.consumption_data.loc[load.consumption_data["Time(Local)"] == hour]
    

In [ ]:
def get_possible_activation_income(hour : pd.Timestamp, load : meters.PowerMeter, market : markets.ReserveMarket, direction : str):
    """ Returns the possible income for a given hour, load and activation market
     income for activating portfolio p that was bid into cap market m at hour h
    """
    # FCR-D => RK pris
    # FCR-N => ingen pris
    # RKOM => RK pris
    # aFRR => ?
     
    if market.name.isin(["FCR-D", "RKOM"]):
        return get_possible_rk_income(hour, load, market, direction)
    else:
        return 0
    

In [ ]:
def get_possible_fee():
    """fee for not activating portfolio p in cap market m that recieved an activation signal in hour h
    """
    # Avkortning (EUR) = avviksfaktor A x pris (EUR/MWh) x manglende effektvolum (MWh), A= 2
    

In [ ]:
def get_available_flex_volume(meter : meters.PowerMeter, hour: pd.Timestamp, ):
    return meter.flex_volume.loc[meter.flex_volume["Time(Local)"] == hour].values[0]

In [ ]:
def get_market_volume(market : markets.ReserveMarket, hour: pd.Timestamp):
    return market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0]

In [ ]:
def get_possible_duration_for_meter(meter : meters.PowerMeter, hour : pd.Timestamp):
    """ returns the number of hours a meter can durate from a given hour

    Args:
        meter (meters.PowerMeter): the given power meter
        hour (pd.Timestamp): the given hour

    Returns:
        int: number of minutes the meter can durate
    """
    # A meter can durate for the same amount of hours as it has a higher or equal flex volume in the following hours
    following_hours = meter.flex_volume.loc[meter.flex_volume["Time(Local)"] >= hour]
    # find how many hours where the value is higher than the value for the current hour
    amount_of_hours = 0
    for value in following_hours["value"][1:]:
        if value >= meter.flex_volume["value"].loc[meter.flex_volume["Time(Local)"] == hour].values[0]:
            amount_of_hours += 1
        else:
            return amount_of_hours * 60

In [ ]:
#def add_load_to_portfolio(portfolio : Portfolio, load : meters.PowerMeter, portfolio_list : list[Portfolio]):
    

In [ ]:
get_possible_duration_for_meter(power_meter_dict[L[0].meter_id], pd.Timestamp(year = 2022, month = 6, day = 26, hour = 4, tz = "Europe/Oslo"))

# Optimization model

In [ ]:
opt_model = Model(name="Optimization Model")

## Decision Variables

In [ ]:
# Add multiple binary decision variables
x = opt_model.addVars(len(L) * len(H), vtype = GRB.BINARY, name = "x") # load l is in portfolio p at hour h
z = opt_model.addVars(len(M) * len(H), vtype = GRB.BINARY, name = "z") # portfolio p is bid to market m at hour h
#w = opt_model.addVars(len(M) * len(H), vtype = GRB.BINARY, name = "w") # portfolio p recieves activation signal in market m at hour h
#y = opt_model.addVars(len(M) * len(H), vtype = GRB.BINARY, name = "y") # portfolio p is activated in market m at hour h

# Continous variables
d = opt_model.addVars(len(M) * len(H), vtype = GRB.CONTINUOUS, name = "d") # flex volume bid in to market m in hour h


## Constraints

In [ ]:
# one portfolio per meter
opt_model.addConstrs((sum(x[l,p, h] for p in P) <= 1 for l in L for h in H), name = "one_portfolio_per_meter")
# one market per portfolio
opt_model.addConstrs((sum(z[m,p, h] for m in M) <= 1 for p in P for h in H), name = "one_market_per_portfolio")
# minimum volume for up markets
opt_model.addConstrs(((p.flex_volume_up - m.min_volume) * z[m,p,h] >= 0  for m in M for p in P for h in H), name = "min_bid_vol_up")
# minimum volume for down markets
opt_model.addConstrs(((p.flex_volume_down - m.min_volume) * z[m,p,h] >= 0  for m in M for p in P for h in H), name = "min_bid_vol_down")
# response time faster than market criteria
opt_model.addConstrs(((p.response_time - m.response_time) * z[p,m,h] <= 0 for m in M for p in P for h in H), name = "response_time")
# sleep time shorter than market criteria
opt_model.addConstrs(((p.sleep_time - m.sleep_time ) * z[p,m,h] <= 0 for m in M for p in P for h in H), name = "sleep_time")
# can only be activated in one market
#opt_model.addConstrs((sum(y[p,m,h]*w[p,m,h] for m in Mc) <= 1 for p in P for h in H), name = "only activated in one market")
# can only be activated if bid
#opt_model.addConstrs((w[p,m,h] <= z[p,m,h] for m in Mc for p in P for h in H), name = "only activated if bid")
# can only be activated if signal
#opt_model.addConstrs((w[p,m,h] <= y[p,m,h] for m in Mc for p in P for h in H), name = "only activated if signal")


In [ ]:
#obj_fn = sum(sum( (sum(get_possible_rk_income(h, p, m, m.direction)*z[m,p,h]) for m in Ma) + sum(get_possible_arming_income(h, p, m) * z[p,m,h] for m in Mc) ) for p in P for h in H) - sum(sum(get_possible_activation_income(h, p, m, m.direction) * y[p,m,h] for m in Mc) for p in P for h in H)

obj_fn = sum(sum(sum(get_portfolio_income(p, m, h) * z[p,m,h] for m in M) for p in P) for h in H)

opt_model.setObjective(obj_fn, GRB.MAXIMIZE)

opt_model.print_information()

In [ ]:
opt_model.optimize()

